In [9]:
import ee
import folium
ee.Authenticate()
ee.Initialize()
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())
from IPython.display import Image




Successfully saved authorization token.
NASADEM: NASA NASADEM Digital Elevation 30m


In [35]:
ee.Initialize()

def mask_s2_clouds(image):
    """Masks clouds in a Sentinel-2 image using the QA band.

    Args:
        image (ee.Image): A Sentinel-2 image.

    Returns:
        ee.Image: A cloud-masked Sentinel-2 image.
    """
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = (
        qa.bitwiseAnd(cloud_bit_mask)
        .eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )

    # Create a binary mask where clouds are 1 and non-clouds are 0
    cloud_mask = mask.Not()

    # Make clouds black and non-clouds unchanged
    return image.where(cloud_mask, 0).divide(10000)

point1 = ee.Geometry.Point([-48.503889, -1.455833])

s2Sr = ee.ImageCollection('COPERNICUS/S2_SR')

img1 = s2Sr.filterBounds(point1).first()

# Aplica a função mask_s2_clouds à imagem
img1 = mask_s2_clouds(img1)

map_center = [point1.getInfo()['coordinates'][1], point1.getInfo()['coordinates'][0]]
map1 = folium.Map(location=map_center, zoom_start=11)

map_id = img1.getMapId({'min': 0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']})

folium.TileLayer(
    tiles=map_id['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='Sentinel 2 RGB L2A - Point 1'
).add_to(map1)

folium.LayerControl().add_to(map1)

map1